In [ ]:
CATALOGO_DESTINO = "spotify_analytics"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_tracks_invalidos"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### Configuração

Define catálogo, schema e nome da tabela de auditoria para tracks inválidos.

### DDL - Tabela de Auditoria (Tracks Inválidos)

Armazena registros rejeitados na validação de qualidade com:
- Todos os campos da tabela principal
- **Flags de validação** para diagnóstico:
  - flag_id_valido
  - flag_nome_valido
  - flag_duracao_valida
  - flag_qualidade (OK/ERRO)

Permite rastreamento e correção de problemas de qualidade.

In [ ]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    track_id STRING COMMENT 'ID da track (pode ser nulo em registros inválidos)',
    track_name STRING,
    album_name STRING,
    album_type STRING,
    release_date DATE,
    release_year INT,
    popularity INT,
    duration_ms BIGINT,
    explicit BOOLEAN,
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    flag_id_valido BOOLEAN COMMENT 'Valida se track_id está preenchido',
    flag_nome_valido BOOLEAN COMMENT 'Valida se track_name está preenchido e tem mais de 1 caractere',
    flag_duracao_valida BOOLEAN COMMENT 'Valida se duration_ms > 0',
    flag_qualidade STRING COMMENT 'Status geral: OK ou ERRO'
)
USING DELTA
COMMENT 'Tabela de auditoria: registros de tracks que falharam na validação de qualidade'
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'quality' = 'audit',
    'purpose' = 'data_quality_monitoring'
);
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")